# **Import data**

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_scheduler, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import random, os
from google.colab import drive

In [2]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 18.1 MB/s eta 0:00:00


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
torch.cuda.empty_cache()

In [5]:
DATA_PATH = '/content/drive/MyDrive/DevWorkshop_data'

TRAIN_PATH = DATA_PATH + '/train.csv'
TEST_PATH = DATA_PATH + '/test.csv'

In [6]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

# **Preprocess data**

## **Check data**

In [7]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [8]:
df_train.head()

,topic_id,PID,relevance,title,abstract
0,CD007431,7072537,0,Lumbar spondylolisthesis. Clinical syndrome an...,"The paper gives a survey, based on literature ..."
1,CD007431,8748845,0,The C-reactive protein for detection of early ...,The tendency for short hospitalization after l...
2,CD007431,3819738,0,Pain in sciatica depresses lower limb nocicept...,The inhibitory effects of acute pain produced ...
3,CD007431,7941692,0,[Satisfaction following automated percutaneous...,182 patients assessed their condition after au...
4,CD007431,16261104,0,Adjacent segment degeneration at T1-T2 present...,A case report of a T1-T2 herniated disc adjace...


Amount of unique topics - train

In [9]:
uq_topics = df_train['topic_id'].unique()

len(uq_topics)

99

Amount of unique topics - test

In [10]:
uq_topics_test = df_test['topic_id'].unique()

len(uq_topics_test)

28

Topics from train that are also in test

In [11]:
[x for x in uq_topics if x in uq_topics_test]

['CD011686', 'CD011571', 'CD012164']

So for each topic there is need to create train_test split

Topics with most positive 'relevance':

In [12]:
df_train.groupby('topic_id').sum('relevance').sort_values(by='relevance', ascending=False)

,PID,relevance
topic_id,,
CD011975,67435351855,414
CD010213,158206893401,402
CD012599,66093127885,402
CD009925,43777216611,314
CD011984,67413914183,307
...,...,...
CD006715,22770758,1
CD011549,78868246171,1
CD011571,99935044,1


In [13]:
len(df_train.groupby('topic_id').sum('relevance')[df_train.groupby('topic_id').sum('relevance')['relevance'] >= 70])

23

In [14]:
top_topics = df_train.groupby('topic_id').sum('relevance')[
                df_train.groupby('topic_id').sum('relevance')['relevance'] >= 70
                ].sort_values(by='relevance', ascending=False).index

## **Dataset and DataLoader**

In [15]:
def split_Xy(topic, df=df_train):
  df_topic = df[df['topic_id'] == topic]
  df_topic.dropna(inplace=True)
  df_topic.reset_index(drop=True, inplace=True)

  X = df_topic.drop(columns=['relevance', 'topic_id', 'PID'], axis=1) # we leave only topic and abstract
  y = df_topic['relevance']

  return X, y

Now checking optimal max_length for this dataset

In [16]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def calculate_max_length(X):
  title_lens = [len(tokenizer.tokenize(t)) for t in X.iloc[:, 0]]
  abstract_lens = [len(tokenizer.tokenize(a)) for a in X.iloc[:, 1]]

  print("Title stats:", np.percentile(title_lens, [50, 75, 90, 95]))
  print("Abstract stats:", np.percentile(abstract_lens, [50, 75, 90, 95]))
  print("Total stats:", np.percentile(title_lens + abstract_lens, [50, 75, 90, 95]))


for topic in top_topics[:5]:
  print(topic)
  X, y = split_Xy(topic)
  calculate_max_length(X)
  print()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

CD011975
Title stats: [21. 26. 33. 38.]
Abstract stats: [151.  271.  353.1 403. ]
Total stats: [ 32.   151.   296.   353.05]

CD010213
Title stats: [22. 28. 34. 39.]
Abstract stats: [145. 252. 349. 413.]
Total stats: [ 36. 145. 280. 349.]

CD012599
Title stats: [21. 26. 33. 38.]
Abstract stats: [154.   271.75 354.3  403.15]
Total stats: [ 32.   154.   297.   354.25]

CD009925
Title stats: [20. 26. 33. 38.]
Abstract stats: [179. 285. 364. 417.]
Total stats: [ 33. 179. 306. 364.]

CD011984
Title stats: [21. 26. 33. 38.]
Abstract stats: [151.  271.  353.2 403. ]
Total stats: [ 32.   151.   296.   353.15]



In [18]:
class TextDataset(Dataset):
    def __init__(self, X, y, tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'), max_length=350):
        self.tokenizer = tokenizer
        self.max_length = max_length

        topics = X['title'].astype(str).tolist()
        abstracts = X['abstract'].astype(str).tolist()

        encodings = self.tokenizer(
            topics,
            abstracts,
            truncation=True,
            return_overflowing_tokens=False,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']
        self.token_type_ids = encodings['token_type_ids']
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'token_type_ids': self.token_type_ids[idx],
            'labels': self.y[idx]
        }

In [19]:
def preprocess(topic, df=df_train):
  set_seed(42)

  X, y = split_Xy(topic, df)
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

  train_dataset = TextDataset(X_train, y_train)
  test_dataset = TextDataset(X_test, y_test)

  train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

  return train_dataset, test_dataset, train_loader, test_loader

# **Model**

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        x = outputs.pooler_output
        x = self.dropout(x)
        x = self.fc(x)

        return x.squeeze(-1)

# **Training**

In [21]:
NUM_OF_EPOCHS = 10

In [22]:
def prepare_for_training(train_dataset, train_loader, lr=2e-5):
  set_seed(42)

  model = BERTClassifier()
  optimizer = AdamW(model.parameters(), lr=lr)
  loss_fn = nn.BCEWithLogitsLoss()  # For binary classification

  total_steps = len(train_loader) * NUM_OF_EPOCHS
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=int(0.1 * total_steps),
      num_training_steps=total_steps
  )

  y_train = train_dataset.y
  num_pos = (y_train == 1).sum()
  num_neg = (y_train == 0).sum()
  pos_weight = torch.tensor([num_neg / num_pos], dtype=torch.float32)

  loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)
  loss_fn = loss_fn.to(device)

  return model, optimizer, loss_fn, scheduler, device

In [23]:
def train(model, optimizer, loss_fn, scheduler, device, train_loader, num_epochs=NUM_OF_EPOCHS):
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")

    return model

# **Evaluation**

In [24]:
def evaluate(model, test_loader, device, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, token_type_ids)
            probs = torch.sigmoid(outputs)  # [batch_size]

            all_preds.append(probs.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # # Binary predictions
    # preds_binary = (all_preds >= threshold).int()

    # # Metrics
    # accuracy = accuracy_score(all_labels, preds_binary)
    # f1 = f1_score(all_labels, preds_binary)
    # precision = precision_score(all_labels, preds_binary)
    # recall = recall_score(all_labels, preds_binary)

    thresholds = [0.1 * i for i in range(1, 10)]
    for t in thresholds:
        preds_binary = (all_preds >= t).int()
        f1 = f1_score(all_labels, preds_binary)
        precision = precision_score(all_labels, preds_binary, zero_division=0)
        recall = recall_score(all_labels, preds_binary, zero_division=0)
        accuracy = accuracy_score(all_labels, preds_binary)
        print(f"Threshold: {t:.1f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | Accuracy: {accuracy:.4f}")


    # print({
    #     "accuracy": accuracy,
    #     "f1_score": f1,
    #     "precision": precision,
    #     "recall": recall
    # })

# **Whole pipeline**

In [25]:
def train_and_evaluate(topic, df=df_train):
  set_seed(42)

  train_dataset, test_dataset, train_loader, test_loader = preprocess(topic, df)

  model, optimizer, loss_fn, scheduler, device = prepare_for_training(train_dataset, train_loader)

  model = train(model, optimizer, loss_fn, scheduler, device, train_loader)

  yield model, test_loader, device

  save_dir = '/content/drive/MyDrive/DevWorkshop_data/models'
  os.makedirs(save_dir, exist_ok=True)

  save_path = os.path.join(save_dir, f"bert_classifier_{topic}.pt")
  torch.save(model.state_dict(), save_path)
  print(f"Model saved to: {save_path}")

  evaluate(model, test_loader, device)

# **TESTS**

In [101]:
top_topics[0]

'CD011975'

In [26]:
# IGNORE ERRORS
import transformers
transformers.logging.set_verbosity_error()

In [103]:
model, test_loader, device = train_and_evaluate(top_topics[0])

Epoch 1: 100%|██████████| 218/218 [03:21<00:00,  1.08it/s]


Epoch 1 | Loss: 1.1369


Epoch 2: 100%|██████████| 218/218 [03:21<00:00,  1.08it/s]


Epoch 2 | Loss: 0.7778


Epoch 3: 100%|██████████| 218/218 [03:21<00:00,  1.08it/s]


...

Epoch 9 | Loss: 0.1089


Epoch 10: 100%|██████████| 218/218 [03:21<00:00,  1.08it/s]


Epoch 10 | Loss: 0.0881
Model saved to: /content/drive/MyDrive/DevWorkshop_data/models/bert_classifier_CD011975.pt


Evaluating: 100%|██████████| 73/73 [00:22<00:00,  3.30it/s]


UnboundLocalError: cannot access local variable 'accuracy' where it is not associated with a value

In [28]:
#load model
train_dataset, test_dataset, train_loader, test_loader = preprocess(top_topics[0])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTClassifier()
model.load_state_dict(torch.load(
    '/content/drive/MyDrive/DevWorkshop_data/models/bert_classifier_CD011975.pt',
    map_location=torch.device('cpu')
))
model = model.to(device)

evaluate(model, test_loader, device)

Evaluating: 100%|██████████| 73/73 [24:58<00:00, 20.53s/it]

Threshold: 0.1 | F1: 0.3952 | Precision: 0.3203 | Recall: 0.5158 | Accuracy: 0.8707
Threshold: 0.2 | F1: 0.3966 | Precision: 0.3358 | Recall: 0.4842 | Accuracy: 0.8793
Threshold: 0.3 | F1: 0.4054 | Precision: 0.3543 | Recall: 0.4737 | Accuracy: 0.8862
Threshold: 0.4 | F1: 0.4055 | Precision: 0.3607 | Recall: 0.4632 | Accuracy: 0.8888
Threshold: 0.5 | F1: 0.3942 | Precision: 0.3628 | Recall: 0.4316 | Accuracy: 0.8914
Threshold: 0.6 | F1: 0.4080 | Precision: 0.3868 | Recall: 0.4316 | Accuracy: 0.8974
Threshold: 0.7 | F1: 0.4205 | Precision: 0.4100 | Recall: 0.4316 | Accuracy: 0.9026
Threshold: 0.8 | F1: 0.4205 | Precision: 0.4568 | Recall: 0.3895 | Accuracy: 0.9121
Threshold: 0.9 | F1: 0.4096 | Precision: 0.4789 | Recall: 0.3579 | Accuracy: 0.9155
